# Latin-Amazigh Section (Modelling)

## Data processing and Augmentation

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras import layers, models, optimizers, backend as K

In [1]:
# Function to load images with quality checks
def load_images(image_folder):
    images = []
    labels = []
    for label in os.listdir(image_folder):
        class_folder = os.path.join(image_folder, label)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            try:
                with Image.open(img_path) as img:
                    # Convert to grayscale and resize
                    img = img.convert('L').resize((28, 28))
                    img_array = np.array(img, dtype=np.float32) / 255.0
                    # Quality control: Check for low contrast images
                    if img_array.std() > 0.05: 
                        images.append(img_array.reshape(28, 28, 1))
                        labels.append(label)
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
                continue
    return np.array(images), labels


latin_path = "C:\\Users\\bouad\\OneDrive\\Bureau\\Amazigh NLP\\MNIST-BERBER\\Latin-version\\imgL"
latin_images, latin_labels = load_images(latin_path)
encoder = LabelEncoder()
latin_labels_encoded = encoder.fit_transform(latin_labels)
num_classes = len(encoder.classes_)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(latin_images, latin_labels_encoded, test_size=0.2, random_state=42)

# Data Augmentation
data_augmentation = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

# Training the model with augmented data
train_generator = data_augmentation.flow(X_train, to_categorical(y_train), batch_size=32)


C:\Users\bouad\anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Capsul Network Model

In [2]:
def squash(x, axis=-1):
    """
    Squashing function to ensure that vectors have magnitudes between 0 and 1.
    """
    s_squared_norm = K.sum(K.square(x), axis=axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * x

def build_capsule_network(input_shape, num_classes):
    input_layer = layers.Input(shape=input_shape)

    # Convolutional layer to extract features
    conv_layer = layers.Conv2D(64, (9, 9), strides=(1, 1), padding='valid', activation='relu')(input_layer)

    primary_caps = layers.Conv2D(32, (9, 9), strides=(2, 2), padding='valid', activation='relu')(conv_layer)
    primary_caps_shape = K.int_shape(primary_caps)
    num_capsules = primary_caps_shape[1] * primary_caps_shape[2] * 32 // 8  # Calculate total number of capsules

    # Reshaping 
    primary_caps = layers.Reshape((num_capsules, 8))(primary_caps)
    primary_caps = layers.Lambda(lambda x: squash(x))(primary_caps)

    digit_caps = layers.Dense(num_classes * 16, activation='relu')(primary_caps)  # Output size: [batch_size, num_capsules, num_classes * 16]

    digit_caps = layers.Reshape((-1, num_classes, 16))(digit_caps)
    output_caps = layers.Lambda(lambda x: squash(x))(digit_caps)

    # Calculating the length of output capsules to match the true labels shape
    y_pred = layers.Lambda(lambda z: K.sqrt(K.sum(K.square(z), axis=-1)))(output_caps)  # Output shape: (batch_size, num_capsules, num_classes)

    # Reducing the dimensionality
    y_pred = layers.Lambda(lambda x: K.max(x, axis=1))(y_pred)

    # Add Decoder network for reconstruction
    decoder_input = layers.Input(shape=(num_classes, 16))
    flat_caps = layers.Flatten()(decoder_input)

    num_elements = int(np.prod(input_shape))  # Convert to integer

    decoder = models.Sequential([
        layers.Input(shape=(num_classes * 16,)),  
        layers.Dense(512, activation='relu'),
        layers.Dense(1024, activation='relu'),
        layers.Dense(num_elements, activation='sigmoid'),
        layers.Reshape(input_shape)
    ])

    decoder_output = decoder(flat_caps)

    # Adding Full model with two inputs and two outputs
    capsnet = models.Model(inputs=[input_layer, decoder_input], outputs=[y_pred, decoder_output])
    capsnet.compile(optimizer=optimizers.Adam(learning_rate=1e-3), 
                    loss=['categorical_crossentropy', 'mse'], 
                    metrics=[['accuracy'], ['mse']])  

    return capsnet

input_shape = (28, 28, 1)
num_classes = 33  # Adjust to the number of classes in your dataset
capsule_model = build_capsule_network(input_shape, num_classes)

# Preparing the training data for the capsule network
X_train_reshaped = X_train.reshape(X_train.shape[0], 28, 28, 1)
y_train_categorical = to_categorical(y_train, num_classes)

decoder_input_data = np.zeros((X_train.shape[0], num_classes, 16))

# Fitting the model using both inputs
capsule_model.fit([X_train_reshaped, decoder_input_data], [y_train_categorical, X_train_reshaped], epochs=10, batch_size=16,
                  validation_data=([X_test.reshape(X_test.shape[0], 28, 28, 1), np.zeros((X_test.shape[0], num_classes, 16))], 
                                   [to_categorical(y_test, num_classes), X_test.reshape(X_test.shape[0], 28, 28, 1)]))



Epoch 1/10
12400/12400 ━━━━━━━━━━━━━━━━━━━━ 494s 39ms/step - lambda_3_accuracy: 0.0348 - loss: nan - sequential_mse: 0.1116 - val_lambda_3_accuracy: 0.0298 - val_loss: nan - val_sequential_mse: 0.0872
Epoch 2/10
12400/12400 ━━━━━━━━━━━━━━━━━━━━ 483s 39ms/step - lambda_3_accuracy: 0.0314 - loss: nan - sequential_mse: 0.0871 - val_lambda_3_accuracy: 0.0298 - val_loss: nan - val_sequential_mse: 0.0871
Epoch 3/10
12400/12400 ━━━━━━━━━━━━━━━━━━━━ 474s 37ms/step - lambda_3_accuracy: 0.0309 - loss: nan - sequential_mse: 0.0871 - val_lambda_3_accuracy: 0.0298 - val_loss: nan - val_sequential_mse: 0.0871
Epoch 4/10
12400/12400 ━━━━━━━━━━━━━━━━━━━━ 468s 38ms/step - lambda_3_accuracy: 0.0314 - loss: nan - sequential_mse: 0.0870 - val_lambda_3_accuracy: 0.0298 - val_loss: nan - val_sequential_mse: 0.0871
Epoch 5/10
12400/12400 ━━━━━━━━━━━━━━━━━━━━ 457s 37ms/step - lambda_3_accuracy: 0.0311 - loss: nan - sequential_mse: 0.0871 - val_lambda_3_accuracy: 0.0298 - val_loss: nan - val_sequential_mse: 0

## Vision Transformer (VIT) Model Using EfficientNetB0

In [8]:
# Resizing images to 32x32
X_train_resized = tf.image.resize(X_train, [32, 32])
X_test_resized = tf.image.resize(X_test, [32, 32])

def build_vit_model(input_shape, num_classes):
    # Using a pre-trained EfficientNetB0 model
    base_model = EfficientNetB0(weights=None, include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output_layer = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

vit_model = build_vit_model((32, 32, 3), num_classes)
vit_model.fit(X_train_resized, to_categorical(y_train), epochs=5, validation_data=(X_test_resized, to_categorical(y_test)))


Epoch 1/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 2267s 340ms/step - accuracy: 0.6929 - loss: 1.1069 - val_accuracy: 0.9497 - val_loss: 0.1628
Epoch 2/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 2123s 342ms/step - accuracy: 0.9330 - loss: 0.2655 - val_accuracy: 0.9716 - val_loss: 0.0978
Epoch 3/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 2114s 341ms/step - accuracy: 0.9193 - loss: 0.2899 - val_accuracy: 0.8940 - val_loss: 0.3589
Epoch 4/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 2118s 340ms/step - accuracy: 0.9159 - loss: 0.2976 - val_accuracy: 0.9444 - val_loss: 0.1904
Epoch 5/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 2243s 359ms/step - accuracy: 0.8885 - loss: 0.3871 - val_accuracy: 0.9182 - val_loss: 0.2822


## LeNet-5 Model ( CNN ArchitectureType)

In [4]:
def build_lenet5(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(6, (5, 5), activation='tanh', input_shape=input_shape, padding='same'),
        layers.AveragePooling2D(pool_size=(2, 2)), 
        layers.Conv2D(16, (5, 5), activation='tanh'),
        layers.AveragePooling2D(pool_size=(2, 2)), 
        layers.Flatten(),
        layers.Dense(120, activation='tanh'),
        layers.Dense(84, activation='tanh'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

lenet5_model = build_lenet5((28, 28, 1), num_classes)
lenet5_model.fit(X_train_reshaped, y_train_categorical, epochs=5, 
                 validation_data=(X_test.reshape(X_test.shape[0], 28, 28, 1), to_categorical(y_test, num_classes)))


C:\Users\bouad\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 81s 12ms/step - accuracy: 0.8631 - loss: 0.5034 - val_accuracy: 0.9660 - val_loss: 0.1180
Epoch 2/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 72s 12ms/step - accuracy: 0.9707 - loss: 0.0984 - val_accuracy: 0.9712 - val_loss: 0.0959
Epoch 3/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 75s 12ms/step - accuracy: 0.9780 - loss: 0.0752 - val_accuracy: 0.9733 - val_loss: 0.0931
Epoch 4/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 86s 14ms/step - accuracy: 0.9813 - loss: 0.0632 - val_accuracy: 0.9756 - val_loss: 0.0836
Epoch 5/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - accuracy: 0.9830 - loss: 0.0565 - val_accuracy: 0.9784 - val_loss: 0.0728


## Multi-Layer Perceptron (MLP) Model

In [5]:
def build_mlp(input_shape, num_classes):
    model = models.Sequential([
        layers.Flatten(input_shape=input_shape),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

mlp_model = build_mlp((28, 28, 1), num_classes)
mlp_model.fit(X_train_reshaped, y_train_categorical, epochs=5, 
              validation_data=(X_test.reshape(X_test.shape[0], 28, 28, 1), to_categorical(y_test, num_classes)))


C:\Users\bouad\anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 48s 7ms/step - accuracy: 0.8425 - loss: 0.5657 - val_accuracy: 0.9430 - val_loss: 0.1958
Epoch 2/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 37s 6ms/step - accuracy: 0.9531 - loss: 0.1594 - val_accuracy: 0.9558 - val_loss: 0.1464
Epoch 3/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 41s 7ms/step - accuracy: 0.9653 - loss: 0.1181 - val_accuracy: 0.9637 - val_loss: 0.1263
Epoch 4/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 45s 7ms/step - accuracy: 0.9696 - loss: 0.0998 - val_accuracy: 0.9665 - val_loss: 0.1176
Epoch 5/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 71s 5ms/step - accuracy: 0.9741 - loss: 0.0875 - val_accuracy: 0.9682 - val_loss: 0.1104


## VGG-Like Small Network (CNN Architecture Type)

In [6]:
def build_vgg_like(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

vgg_like_model = build_vgg_like((28, 28, 1), num_classes)
vgg_like_model.fit(X_train_reshaped, y_train_categorical, epochs=5, 
                   validation_data=(X_test.reshape(X_test.shape[0], 28, 28, 1), to_categorical(y_test, num_classes)))


Epoch 1/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 321s 51ms/step - accuracy: 0.9231 - loss: 0.2768 - val_accuracy: 0.9845 - val_loss: 0.0526
Epoch 2/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 342s 54ms/step - accuracy: 0.9868 - loss: 0.0442 - val_accuracy: 0.9857 - val_loss: 0.0450
Epoch 3/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 314s 51ms/step - accuracy: 0.9912 - loss: 0.0280 - val_accuracy: 0.9894 - val_loss: 0.0354
Epoch 4/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 323s 51ms/step - accuracy: 0.9939 - loss: 0.0193 - val_accuracy: 0.9884 - val_loss: 0.0414
Epoch 5/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 335s 53ms/step - accuracy: 0.9950 - loss: 0.0157 - val_accuracy: 0.9897 - val_loss: 0.0440


## ResNet (Residual Network) Model

In [7]:
def residual_block(x, filters):
    """
    A residual block that adds a shortcut connection to the output of two convolutional layers.
    If the number of filters changes, a 1x1 convolution is applied to the shortcut to match the shape.
    """
    shortcut = x

    # Main path
    x = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)

    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), padding='same')(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def build_resnet_small(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    
    # Adding residual blocks
    x = residual_block(x, 32)
    x = layers.MaxPooling2D((2, 2))(x)
    x = residual_block(x, 64) 
    x = layers.MaxPooling2D((2, 2))(x)
    
    # Flattening and adding dense layers
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Building the model
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

resnet_model = build_resnet_small((28, 28, 1), num_classes)
resnet_model.fit(X_train_reshaped, y_train_categorical, epochs=5, 
                 validation_data=(X_test.reshape(X_test.shape[0], 28, 28, 1), to_categorical(y_test, num_classes)))


Epoch 1/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 436s 69ms/step - accuracy: 0.9218 - loss: 0.2812 - val_accuracy: 0.9840 - val_loss: 0.0560
Epoch 2/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 443s 69ms/step - accuracy: 0.9867 - loss: 0.0466 - val_accuracy: 0.9876 - val_loss: 0.0425
Epoch 3/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 451s 71ms/step - accuracy: 0.9906 - loss: 0.0319 - val_accuracy: 0.9871 - val_loss: 0.0441
Epoch 4/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 425s 68ms/step - accuracy: 0.9924 - loss: 0.0233 - val_accuracy: 0.9892 - val_loss: 0.0412
Epoch 5/5
6200/6200 ━━━━━━━━━━━━━━━━━━━━ 457s 71ms/step - accuracy: 0.9942 - loss: 0.0178 - val_accuracy: 0.9862 - val_loss: 0.0485
